In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
# 1. SPECIFY YOUR TOTALS HERE
# Format: {Year: [Consumer goods, Materials for consumer, Capital goods, Materials for capital]}
imp_totals = {
    '2020': [540221, 4160831, 691419, 49187],
    '2021': [690414, 5177557, 707580, 57501],
    '2022': [1008356, 7039390, 830670, 61101],
    '2023': [1213299, 7029093, 872121, 80835],
    '2024': [1157502, 7359638, 782612, 56716]
}

exp_totals = {
    '2020': [2524550, 208676, 74927, 3515],
    '2021': [3118774, 298894, 60308, 1914],
    '2022': [4121727, 365975, 81858, 1793],
    '2023': [5148940, 400741, 81548, 500],
    '2024': [5774683, 479787, 68693, 1925]
}

In [3]:
# 2. CONFIGURATION
years = ['2020', '2021', '2022', '2023', '2024']
compositions = ['Consumer goods', 'Materials for consumer goods', 'Capital goods', 'Materials for capital goods']
accounts = ['Private', 'Government', 'Semi-government']
routes = ['Sea', 'Air', 'Land']

# Country Lists provided by you
import_countries = ['China', 'India', 'Indonesia', 'Malaysia', 'Hong Kong', 'USA', 'Japan', 'Canada', 'Australia', 'Korea']
export_countries = ['USA', 'Germany', 'UK', 'Singapore', 'Spain', 'France', 'Italy', 'India', 'Japan', 'Canada']
unique_countries = sorted(list(set(import_countries + export_countries)))

# Weights for distribution (Estimates)
imp_w = {'China': 0.25, 'India': 0.20, 'Indonesia': 0.10, 'Malaysia': 0.08, 'Hong Kong': 0.07, 'USA': 0.10, 'Japan': 0.10, 'Canada': 0.04, 'Australia': 0.03, 'Korea': 0.03}
exp_w = {'USA': 0.25, 'Germany': 0.15, 'UK': 0.10, 'Singapore': 0.10, 'Spain': 0.10, 'France': 0.10, 'Italy': 0.10, 'India': 0.05, 'Japan': 0.03, 'Canada': 0.02}
rte_w = {'Sea': 0.85, 'Air': 0.05, 'Land': 0.10}
acc_w = {'Private': 0.90, 'Government': 0.07, 'Semi-government': 0.03}

In [4]:
# 3. GENERATE GRID
grid = list(itertools.product(years, unique_countries, compositions, accounts, routes))
df = pd.DataFrame(grid, columns=['Year', 'Country', 'Composition', 'Account', 'Route'])
df['Import Value'] = 0.0
df['Export Value'] = 0.0

In [5]:
# 4. DISTRIBUTION LOGIC
for yr in years:
    for i, comp in enumerate(compositions):
        # Imports
        mask = (df['Year'] == yr) & (df['Composition'] == comp)
        total_i = imp_totals[yr][i]
        
        weights_i = df[mask].apply(lambda r: imp_w.get(r['Country'], 0) * rte_w[r['Route']] * acc_w[r['Account']], axis=1)
        if weights_i.sum() > 0:
            df.loc[mask, 'Import Value'] = (weights_i / weights_i.sum() * total_i).round(2)

        # Exports
        total_e = exp_totals[yr][i]
        weights_e = df[mask].apply(lambda r: exp_w.get(r['Country'], 0) * rte_w[r['Route']] * acc_w[r['Account']], axis=1)
        if weights_e.sum() > 0:
            df.loc[mask, 'Export Value'] = (weights_e / weights_e.sum() * total_e).round(2)

In [8]:
print(df.shape)
df.head()

(2880, 7)


,Year,Country,Composition,Account,Route,Import Value,Export Value
0,2020,Australia,Consumer goods,Private,Sea,12398.07,0.0
1,2020,Australia,Consumer goods,Private,Air,729.30,0.0
2,2020,Australia,Consumer goods,Private,Land,1458.60,0.0
3,2020,Australia,Consumer goods,Government,Sea,964.29,0.0
4,2020,Australia,Consumer goods,Government,Air,56.72,0.0


In [9]:
# 5. SAVE
df.to_excel('GranularTradeData.xlsx', index=False)